In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tarfile
import tensorflow as tf

from collections import Counter
from scipy import misc
from PIL import Image
from sklearn.metrics import confusion_matrix
from scipy.io import loadmat
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import InceptionV3

## Transfer learning with InceptionV3

![](images/inceptionv3.png)

In [ ]:
tf.keras.backend.clear_session()

In [ ]:
!wget http://www.robots.ox.ac.uk/~vgg/data/flowers/102/102flowers.tgz -O /tmp/flowers.tgz

!wget http://www.robots.ox.ac.uk/~vgg/data/flowers/102/imagelabels.mat -O /tmp/imagelabels.mat

!wget http://www.robots.ox.ac.uk/~vgg/data/flowers/102/setid.mat -O /tmp/datasplit.mat

In [ ]:
labels = loadmat('/tmp/imagelabels.mat')

assignments = loadmat('/tmp/datasplit.mat')

In [ ]:
labels

In [ ]:
assignments

In [ ]:
labels = labels['labels']

In [ ]:
ROOT_FOLDER = '/tmp/flowers'

In [ ]:
def create_folder_structure(root_folder, labels):
    unique_labels = np.unique(labels)
    subfolders = ('train', 'validation', 'test')
    for subfolder_name in subfolders:
        for label_id in np.unique(labels):
            os.makedirs(f'{root_folder}/{subfolder_name}/{label_id}', exist_ok=True)

In [ ]:
def extract_images_to_folders(assignments, labels):
    train_ids = set(np.squeeze(assignments['tstid']))
    validation_ids = set(np.squeeze(assignments['valid']))
    test_ids = set(np.squeeze(assignments['trnid']))
    
    tar_folder = '/tmp/flowers.tgz'
    with tarfile.open(tar_folder, 'r') as tar:
        for item in tar:
            if item.isfile():
                filename = item.name
                item.name = filename[4:]
                name, _ = os.path.splitext(filename)
                file_id = int(name.split('_')[-1])
                label = labels[file_id - 1]
                if file_id in train_ids:
                    tar.extract(item, f'{ROOT_FOLDER}/train/{label}')
                elif file_id in validation_ids:
                    tar.extract(item, f'{ROOT_FOLDER}/validation/{label}')
                else:
                    tar.extract(item, f'{ROOT_FOLDER}/test/{label}')

In [ ]:
create_folder_structure(ROOT_FOLDER, np.squeeze(labels))

In [ ]:
extract_images_to_folders(assignments, np.squeeze(labels))

In [ ]:
train_datagen = ImageDataGenerator(rescale=1/255.,
                                   rotation_range=15,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

validation_datagen = ImageDataGenerator(rescale=1/255.)

test_datagen = ImageDataGenerator(rescale=1/255.)

In [ ]:
train_dir = f'{ROOT_FOLDER}/train'
validation_dir =  f'{ROOT_FOLDER}/validation'
test_dir =  f'{ROOT_FOLDER}/test'

In [ ]:
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size=32,
                                                    class_mode='categorical', 
                                                    target_size=(150, 150))     


validation_generator = validation_datagen.flow_from_directory(validation_dir,
                                                              batch_size=32,
                                                              class_mode='categorical', 
                                                              target_size=(150, 150))


test_generator = test_datagen.flow_from_directory(test_dir,
                                                  batch_size=32,
                                                  class_mode='categorical', 
                                                  target_size=(150, 150))

In [ ]:
def make_model(size=32, nr_classes=10):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(input_shape=(size, size, 3), filters=16, kernel_size=(3, 3), padding="same", 
                               activation="relu"),
        tf.keras.layers.MaxPooling2D(pool_size=2, strides=2),
        tf.keras.layers.Dropout(rate=0.2),
        tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), padding="same", activation="relu"),
        tf.keras.layers.MaxPooling2D(pool_size=2, strides=2),
        tf.keras.layers.Dropout(rate=0.2),
        tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), padding="same", activation="relu"),
        tf.keras.layers.MaxPooling2D(pool_size=2, strides=2),
        tf.keras.layers.Dropout(rate=0.2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(256, kernel_regularizer=tf.keras.regularizers.l2(0.001), activation="relu"),
        tf.keras.layers.Dense(nr_classes, activation='softmax')  
    ])
    return model

In [ ]:
tf.keras.backend.clear_session()

In [ ]:
model = make_model(size=150, nr_classes=102)

In [ ]:
model.summary()

In [ ]:
LR_INIT = 0.002
BATCH_SIZE = 32
EPOCHS = 2


def lr_scheduler(epoch):
    return LR_INIT * (0.98 ** epoch)


model.compile(optimizer=tf.keras.optimizers.Adamax(learning_rate=LR_INIT),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    callbacks=[tf.keras.callbacks.LearningRateScheduler(lr_scheduler)],
    validation_data=validation_generator,
    verbose=1
)

In [ ]:
tf.keras.backend.clear_session()

In [ ]:
base_model = InceptionV3(input_shape=(150, 150, 3), weights='imagenet', include_top=False)

In [ ]:
base_model.summary()

In [ ]:
tf.keras.backend.clear_session()

In [ ]:
# def build_model():
#     base_model = InceptionV3(input_shape=(150, 150, 3), weights='imagenet', include_top=False)
#     for layer in base_model.layers:
#         layer.trainable = False
        
#     last_layer = base_model.get_layer('mixed9')
#     X = tf.keras.layers.Flatten()(last_layer.output)
#     X = tf.keras.layers.Dense(256, activation='relu')(X)
#     X = tf.keras.layers.Dropout(0.2)(X)   
#     predictions = tf.keras.layers.Dense(102, activation='softmax')(X)
#     model = tf.keras.models.Model(inputs=base_model.input, outputs=predictions)
#     return model

In [ ]:
def build_model():
    base_model = InceptionV3(input_shape=(150, 150, 3), weights='imagenet', include_top=False)
    for layer in base_model.layers:
        layer.trainable = True
        #if isinstance(layer, tf.keras.layers.BatchNormalization):
        #    layer.momentum = 0.8
    
    for layer in base_model.layers[:-50]:
        if not isinstance(layer, tf.keras.layers.BatchNormalization):
            layer.trainable = False
        
    X = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
    predictions = tf.keras.layers.Dense(102, activation='softmax')(X)
    model = tf.keras.models.Model(inputs=base_model.input, outputs=predictions)
    return model

In [ ]:
model = build_model()

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adamax(lr=0.005), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [ ]:
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=20,
    verbose=1)

In [ ]:
model.evaluate(test_generator)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = np.arange(len(acc))

In [ ]:
plt.figure(figsize=(16, 4))
plt.plot(epochs, acc)
plt.plot(epochs, val_acc)
plt.title('Training and validation accuracy')
plt.show()

In [ ]:
plt.figure(figsize=(16, 4))
plt.plot(epochs, loss)
plt.plot(epochs, val_loss)
plt.title('Training and validation loss')
plt.show()

In [ ]:
tf.keras.backend.clear_session()